In [ ]:
import pandas as pd
import seaborn as sns
import os
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
from scipy import signal
import scipy.integrate as integrate
import scipy.fftpack as ff
#!/usr/bin/env python

import numpy as np, os, sys, joblib
from scipy.io import loadmat
from sklearn.impute import SimpleImputer

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pathlib import Path
from numpy import genfromtxt

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Dense, Dropout, TimeDistributed, Flatten
from keras.optimizers import Adam
import keras

import matplotlib.pyplot as plt
%matplotlib inline

# 1. Signal lecture 
#### Please note: 
For easier data analysis and features visualization, please use pandas (since its easier to use). But for the final signal (the one we need to integration), please use numpy format (convert to numpy using "signal.values")

In [31]:
features = f'features.csv'
labels = f'labels.csv'
testing = f'testing.csv'
testing = genfromtxt(testing, delimiter=',')
features = genfromtxt(features, delimiter=',')
labels = genfromtxt(labels, delimiter=',')

In [32]:
# Normalize data
testing = tf.keras.utils.normalize(testing)
features = tf.keras.utils.normalize(features)

In [33]:
print(features.dtype)
print(testing.dtype)
print(labels.dtype)

float64
float64
float64


In [34]:
## Inside the run_12ECG_code we must consider the following structure (1,14)
testing_reshape = testing.reshape(1, -1)
imputer=SimpleImputer().fit(testing_reshape)
testing_reshape = imputer.transform(testing_reshape)
print(testing_reshape.shape)

(1, 14)


In [35]:
# Training set bust consider the next structure
print(features.shape, labels.shape)
4812*14

(4813, 14) (4813, 9)


67368

In [60]:
# Since labels are organized in hot-encoding format and we need the index Number, define decode:
def decode(datum):
    return np.argmax(datum)
label_final = []
    
# Extract hot-encode format to normal format
for i in range(labels.shape[0]):
    label = labels[i]
    decoded_labels = decode(label)
    label_final.append(decoded_labels)
# Convert list to np array
label_final = np.array(label_final)
label_final = np.uint8(label_final)
print(label_final.dtype)

uint8


In [61]:
features.shape

(4813, 14)

### Model 0 - Basic Sequential

In [62]:
model_0 = Sequential()
model_0.add(Dense(12, input_dim = 14, activation="softmax"))
model_0.add(Dense(128, activation = 'softmax'))
model_0.add(Dense(9, activation = 'softmax'))
# Compile the model
model_0.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics=['accuracy'])
# Fit the model
model_0.fit(features,labels, epochs=15, batch_size=10,  verbose=2)

Epoch 1/15
 - 1s - loss: 2.2486 - accuracy: 0.2240
Epoch 2/15
 - 1s - loss: 2.1783 - accuracy: 0.2256
Epoch 3/15
 - 1s - loss: 2.1682 - accuracy: 0.2256
Epoch 4/15
 - 1s - loss: 2.1669 - accuracy: 0.2256
Epoch 5/15
 - 1s - loss: 2.1669 - accuracy: 0.2256
Epoch 6/15
 - 1s - loss: 2.1669 - accuracy: 0.2256
Epoch 7/15
 - 1s - loss: 2.1669 - accuracy: 0.2256
Epoch 8/15
 - 1s - loss: 2.1670 - accuracy: 0.2256
Epoch 9/15
 - 1s - loss: 2.1674 - accuracy: 0.2256
Epoch 10/15
 - 1s - loss: 2.1673 - accuracy: 0.2256
Epoch 11/15
 - 1s - loss: 2.1671 - accuracy: 0.2256
Epoch 12/15
 - 1s - loss: 2.1671 - accuracy: 0.2256
Epoch 13/15
 - 1s - loss: 2.1671 - accuracy: 0.2256
Epoch 14/15
 - 1s - loss: 2.1672 - accuracy: 0.2256
Epoch 15/15
 - 1s - loss: 2.1669 - accuracy: 0.2256


#### Model 1 : branch NN_1

In [63]:
print("training model NN_1")
############# MODEL 1 - NN ######################
model_1 = tf.keras.models.Sequential()
model_1.add(tf.keras.layers.Flatten())
# use 128 neurons & use relu act func
model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
# the final output layer must be Dense and must fit # classifications and must use probability distribution instead of an activation function
model_1.add(tf.keras.layers.Dense(9, activation = tf.nn.softmax))
    
# Compile 
model_1.compile(
optimizer= 'adam' ,# NN intends to minimize losss, not maximize accuracy
loss = 'sparse_categorical_crossentropy',
metrics = ['accuracy']
)
# Fit the model appropiatelly
model_1.fit(features, label_final, epochs=10)
val_loss, val_acc = model_1.evaluate(features, label_final)
print("The model is trained")

training model NN_1
Train on 4813 samples
Epoch 1/10
4813/4813 [==============================] - 2s 415us/sample - loss: 2.0662 - accuracy: 0.2225
Epoch 2/10
4813/4813 [==============================] - 1s 153us/sample - loss: 2.0553 - accuracy: 0.2256
Epoch 3/10
4813/4813 [==============================] - 1s 146us/sample - loss: 2.0552 - accuracy: 0.2256
Epoch 4/10
4813/4813 [==============================] - 1s 136us/sample - loss: 2.0553 - accuracy: 0.2256
Epoch 5/10
4813/4813 [==============================] - 1s 134us/sample - loss: 2.0540 - accuracy: 0.2256
Epoch 6/10
4813/4813 [==============================] - 1s 165us/sample - loss: 2.0514 - accuracy: 0.2256
Epoch 7/10
4813/4813 [==============================] - 1s 177us/sample - loss: 2.0519 - accuracy: 0.2256
Epoch 8/10
4813/4813 [==============================] - 1s 187us/sample - loss: 2.0526 - accuracy: 0.2256
Epoch 9/10
4813/4813 [==============================] - 1s 171us/sample - loss: 2.0519 - accuracy: 0.2256
Epoc

In [64]:
model_1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          multiple                  0         
_________________________________________________________________
dense_14 (Dense)             multiple                  1920
_________________________________________________________________
dense_15 (Dense)             multiple                  16512     
_________________________________________________________________
dense_16 (Dense)             multiple                  16512     
_________________________________________________________________
dense_17 (Dense)             multiple                  16512     
_________________________________________________________________
dense_18 (Dense)             multiple                  16512     
_________________________________________________________________
dense_19 (Dense)             multiple                  16512

In [65]:
# Save model.
model_1.save("NN_1.model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: NN_1.model/assets


In [66]:
print(features.shape, label_final.shape)

(4813, 14) (4813,)


In [67]:
nn_model = tf.keras.models.load_model('NN_1.model/')
nn_model

### current label & score
1. From the baseline code we had that the original model saved three things:`model`, `imputer`, `classes` on a file called `finalized_model.sav` from the joblib library. 
2. The Imputer used in the training code might be affecting the testing set. Imputer replaces np.nan values inside the features and it is saved only once for the entire traininset, therefore it must be saved to be re used in the driver.py code
3. `current_label`: is originally extracted using:

 `current_label = model.predict(feats_reshape)[0]`:  Array of float64 (9,) # array([0., 1., 0., 0., 0., 0., 0., 0., 1.])
 
 `current_label=current_label.astype(int)`:  Array of int32 (9,) # array([0, 1, 0, 0, 0, 0, 0, 0, 1])

 `current_score = model.predict_proba(feats_reshape)`: List of 9 which is converted to np.array # array([0.04, 0.74, 0.02, 0.04, 0.09, 0.02, 0.  , 0.03, 0.72])

 `classes`: This function is stored as well and contains the classes tags:  
 
 classes = ['164884008', '164889003', '164909002', '164931005', '270492004', '284470004', '426783006', '429622005', '59118001']
   

In [81]:
# adding [0] for extracting the values from the array
current_label = nn_model.predict([testing_reshape])[0]
current_label = current_label.astype(int)
current_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

Evidently this will be zero all the time unless prediction is really high. Therefore we can use argmax.

In [84]:
current_label = nn_model.predict(testing_reshape)[0]
current_label

array([0.10544974, 0.16263951, 0.0378426 , 0.04765469, 0.10410184,
       0.0878576 , 0.13401309, 0.11713167, 0.2033093 ], dtype=float32)

In [85]:
# Using the baseline probaility we obtain:
current_score = nn_model.predict_proba(testing_reshape)
current_score

array([[0.10544974, 0.16263951, 0.0378426 , 0.04765469, 0.10410184,
        0.0878576 , 0.13401309, 0.11713167, 0.2033093 ]], dtype=float32)

In [86]:
# Suppose softmax model 
probability_model = tf.keras.Sequential([nn_model,tf.keras.layers.Softmax()])  
current_score = nn_model.predict(testing_reshape)
current_score

array([[0.10544974, 0.16263951, 0.0378426 , 0.04765469, 0.10410184,
        0.0878576 , 0.13401309, 0.11713167, 0.2033093 ]], dtype=float32)

From this results we can infer that nn_model.predict is equivalent to probability_model from Softmax performace

Conclusion

In [87]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

prediction = nn_model.predict([testing_reshape])[0]
print(prediction)
prediction = np.argmax(prediction)
encoded = to_categorical(prediction)
current_label = encoded.astype(int)
print(current_label)

[0.10544974 0.16263951 0.0378426  0.04765469 0.10410184 0.0878576
 0.13401309 0.11713167 0.2033093 ]
[0 0 0 0 0 0 0 0 1]


In [88]:
test = [0.9403204, 0.16651522, 0.02546168, 0.03028586, 0.09770199, 0.08466987, 0.1342014,  0.11385558, 0.9403204]

In [89]:
test = np.array(test)
test

array([0.9403204 , 0.16651522, 0.02546168, 0.03028586, 0.09770199,
       0.08466987, 0.1342014 , 0.11385558, 0.9403204 ])

In [90]:
current_label = test.astype(int)
current_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

#### Model 2: CNN

In [91]:
sequence_size = features.shape[1]
n_features=1
print(features.shape, label_final.shape)

(4813, 14) (4813,)


In [95]:
sequence_size

14

In [97]:
cnn_model = Sequential([
Conv1D(
    filters=1,
    kernel_size=4,
    strides=1,
    input_shape=(sequence_size, n_features),
    padding="same",
    activation="relu"
),
Flatten(),
Dropout(0.5),
Dense(
    9,
    activation="sigmoid",
    name="output",
)
])
optimizer = Adam(lr=0.001)
# Compiling the model
cnn_model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
cnn_model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 14, 1)             5         
_________________________________________________________________
flatten_5 (Flatten)          (None, 14)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14)                0         
_________________________________________________________________
output (Dense)               (None, 9)                 135       
Total params: 140
Trainable params: 140
Non-trainable params: 0
_________________________________________________________________


In [98]:
feat_cnn = np.expand_dims(features, axis=2) 
label_cnn = label_final.reshape(-1,1)
print("features shape: ", feat_cnn.shape)
print("label features: ", label_cnn.shape)

features shape:  (4813, 14, 1)
label features:  (4813, 1)


In [99]:
labels.shape

(4813, 9)

In [100]:
cnn_model.fit(feat_cnn, labels,  epochs=15, batch_size=10,  verbose=2)

Epoch 1/15
 - 1s - loss: 0.5371 - accuracy: 0.8282
Epoch 2/15
 - 1s - loss: 0.3616 - accuracy: 0.8811
Epoch 3/15
 - 1s - loss: 0.3567 - accuracy: 0.8816
Epoch 4/15
 - 1s - loss: 0.3547 - accuracy: 0.8816
Epoch 5/15
 - 1s - loss: 0.3535 - accuracy: 0.8816
Epoch 6/15
 - 1s - loss: 0.3532 - accuracy: 0.8816
Epoch 7/15
 - 1s - loss: 0.3511 - accuracy: 0.8816
Epoch 8/15
 - 1s - loss: 0.3505 - accuracy: 0.8816
Epoch 9/15
 - 1s - loss: 0.3496 - accuracy: 0.8816
Epoch 10/15
 - 1s - loss: 0.3494 - accuracy: 0.8816
Epoch 11/15
 - 1s - loss: 0.3481 - accuracy: 0.8816
Epoch 12/15
 - 1s - loss: 0.3473 - accuracy: 0.8816
Epoch 13/15
 - 1s - loss: 0.3474 - accuracy: 0.8816
Epoch 14/15
 - 1s - loss: 0.3466 - accuracy: 0.8816
Epoch 15/15
 - 1s - loss: 0.3464 - accuracy: 0.8816


In [82]:
testing_reshape_cnn = np.expand_dims(testing_reshape, axis=2) 
testing_reshape_cnn.shape

(1, 14, 1)

In [83]:
# feat_final and labels:  acc_88.16
prediction_cnn = cnn_model.predict([testing_reshape_cnn])[0]

#### CNN + LSTM Model

In [108]:
sequence_size = sequence_size
n_features = 1 
n_subsequences = 2
subsequence_size = int(sequence_size / n_subsequences)

In [109]:
feat_cnn.shape

(4813, 14, 1)

In [115]:
feat_lstm = feat_cnn.reshape(-1, n_subsequences, subsequence_size, n_features)

In [117]:
feat_lstm.shape

(4813, 2, 7, 1)

In [111]:
cnn_lstm_model = Sequential([
    TimeDistributed(
        Conv1D(
            filters=8,
            kernel_size=4,
            strides=1,
            padding="same",
            activation="relu"
        ), 
        input_shape=(n_subsequences, subsequence_size, n_features)
    ),
    TimeDistributed(Flatten()),
    LSTM(units=4),
    Dense(
        1,
        activation="sigmoid",
        name="output",
    )
])

optimizer = Adam(lr=0.001)
# Compiling the model
cnn_lstm_model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
cnn_lstm_model.summary()


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 2, 7, 8)           40        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 2, 56)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4)                 976       
_________________________________________________________________
output (Dense)               (None, 1)                 5         
Total params: 1,021
Trainable params: 1,021
Non-trainable params: 0
_________________________________________________________________


In [119]:
train_params = {
    "batch_size": 128,
    "epochs": 15,
    "verbose": 1,
}

history_cnn_lstm = cnn_lstm_model.fit(feat_lstm, label_final, **train_params)

Epoch 1/1
4813/4813 [==============================] - 2s 419us/step - loss: -3.7503 - accuracy: 0.1627
